#**PFAS Lab Report Extraction Tool**  
####This notebook script will perform extraction of PFAS sample test results from predefined testing lab report formats.    
####Developed for MassDEP by WPI Data Science Graduate Qualifying Project Team - Fall 2021
gr-gqpfall21massdep@wpi.edu

---

###**Report types currently defined:**
*   SGS
*   Con-Test
*   Alpha Analytical  
---

#**Instructions**


###**Google Drive Requirements**  

If hosted on Google Drive, this notebook will be run in the Google Colaboratory (Colab) hosted Jupyter notebook service.  Running this notebook "as is" requires a Google account and access to the current Google Drive folder. 

The notebook expects the current folder to be located in the root of your Google Drive folder.  This can be accomplished without changing the folder location by creating a shortcut. Create a shortcut to the current "PFAS Lab Report Extractor" folder and save the shortcut to your Google Drive root folder, as described below:  

1.   In the Google Drive directory, right click on the current folder "*PFAS Lab Report Extraction Tool/*" and select "***Add Shortcut to Drive***". 
2.   When asked to select where to add the shortcut, highlight "*My Drive*" (without selecting the arrow to open the Drive folder), and click "*Add Shortcut*". 
3.   A shortcut will now be created in your root Google Drive folder.  
  
https://support.google.com/drive/answer/9700156?hl=en&co=GENIE.Platform%3DDesktop  

As long as there is a shortcut to the "PFAS Lab Report Extractor" folder in your Google Drive root folder, the folder that it points to be located anywhere in a Google Drive structure (i.e., it may be copied from its original location in the "GQP Fall 2021/" shared folder to a new location for MassDEP use). Alternatively, the shortcut may be saved a different location if desired, with minor updates to the code to reflect the change.  
 
**Mounting and Autorizing Google Drive Access**  
The notebook will attempt to mount the Google Drive folder, and request access (either via "click" to authorize, or by copying and pasting an authorization string in the notebook cell where indicated. *Authorization must be granted to enable this workbook to run in Google Colab and access the PDF Lab reports on Google Drive.*

The notebook will attempt to mount the Google Drive folder, and request access (either via "click" to authorize, or by copying and pasting an authorization string in the notebook cell where indicated. 

---

###**Alternative Location Requirements**

Instead of the current Google Drive setup, this notebook can be run locally or in another hosted Jupyter environment with minor modifications to certain file location commands.  The folder structure must remain intact (same as the current folder structure).  

---

###**Library Dependencies**  
If using Google Colab, all required python libraries will be installed and loaded by the notebook, as necessary.    
If running locally, all required python libraries must be installed and avaialble on the system, or loaded by the notebook in a virtual environment.   

Future updates to these libraries could result in compatability issues.  If library are encountered, it is possible to "roll back" the version of the required libraries to a previous version (check library documentation for details).  

**extraction_utils**  
The custom file 'extraction_utils.py" must exist in the "*Code/*" folder.  This file constains specialized functions for performing the report extraction tasks on predefined report types. Updates to this file may change what information is extracted from reports, or impact overall execution.  *Always create a backup of this file before modifying its contents*.


**Camelot**:   
PDF table extraction is handled by the "camelot-py" library.   
Current version 0.10.1  
https://camelot-py.readthedocs.io/en/master/  
https://pypi.org/project/camelot-py/  
  

**PyPDF2**:  
PDF text extraction is handled by the PyPDF2 library.  
Current version 1.26.0  
https://pypi.org/project/PyPDF2/  
https://github.com/mstamy2/PyPDF2  

**Additional packages installed:**

Ghostscript >= 0.7  
pdftopng >= 3.4.2.17  
pdfminer.sit-20211012  
tabulate>=0.8.9  
pdftopng >= 0.2.3  

---
###**PFAS Compounds**

The "PFAS_compounds.csv" file contains the names, acronymns, and CAS Numbers for PFAS compounds to be extracted by this tool.  It must be located in the "*Code/*" folder.  Modifying the existing PFAS compound information in this file may have unintended consequences that affect the ability to extract data.  It may be possible to update this file to identify additional PFAS compounds to be extracted. *Always create a backup of this file before modifying its contents*.  
  
The PFAS Lab Report Extraction Tool cannot currently extract compound results other than PFAS.  Modifying this file will not allow other non-PFAS compounds to be extracted.  Doing so would require additional modifications to this notebook and the custom functions located in *extraction_utils.py*.  

---

###**Target Reports**  
Copy folder(s) containing the reports to be extracted into the "*Target Reports/*" folder.  The tool can process more than one folder at a time, if desired.  The name of the folder will be stored in the resulting output file, so it is recommended to name folders in a logical and consistent way.  

Once target report folders have been extracted it is recommended to move them out of the "*Target Reports/*" folder (to avoid having them be extracted again when this code is run again).  Alternatively this code could potentially be modified to exclude previously extracted reports.) 

---

###**Output Data:**  
Output data files will be saved in the "*Output Data/*" folder.  The following three files will be created, and each of their filenames will include the date and time that extraction was completed (EST).

*  **"Extracted PFAS Data - DATE TIME.csv"**
> *All PFAS sample results*
*  **"Report Info - DATE TIME.csv"**  
>*Information on extracted reports and results pages*
*  **"Report Errors - DATE TIME.csv"**  
>*Information on reports and results pages which could not be extracted*

---



#Mount Google Drive

In [ ]:
# Mount Google Drive
# Provide authorization if requested. 
 
from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


# Install and Import Libraries

In [ ]:

!pip install ghostscript
!pip install camelot-py[cv]
!apt install ghostscript python3-tk

from ctypes.util import find_library
print("Ghostscript library:", find_library("gs")) 


     |████████████████████████████████| 40 kB 26 kB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 5.6 MB 22.2 MB/s 
     |████████████████████████████████| 11.7 MB 18.9 MB/s 
     |████████████████████████████████| 3.6 MB 50.5 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=d766a8f08237cc65dc6f2f8080e3933cd26af05f149460604eee3a92e020101b
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  

In [ ]:

import ghostscript
import camelot
import numpy as np
import pandas as pd
import glob
import re
import os
import sys
import importlib
import datetime
import pytz
import warnings

!pip install PyPDF2
import PyPDF2 


In [ ]:

# set main file path. 
# If using Google Drive, a shortcut must be created to the 'PFAS Lab Report Extraction Tool/' folder in the root Google drive folder.  

# modify this path if running in an environment other than Google Drive

main_path = '/gdrive/MyDrive/PFAS Lab Report Extraction Tool/' 
os.chdir(main_path)


In [ ]:

# Import extraction_utils.py from 'Code/' folder

sys.path.append('Code')
import extraction_utils as utils


In [ ]:

# Identify main 'Target Reports/' and 'Output/' folder names

target_path = 'Target Reports/'
output_path = 'Output/'


In [ ]:

# setup output filenames
# date and time will be added to filenames at completion

extracted_data_savefile = 'Extracted PFAS data - '
report_df_savefile = 'Extracted lab report info - '
error_df_savefile = 'Error Reports - '

print('Output filenames:')
print('\n"', extracted_data_savefile, 
      '"\n(All PFAS sample results)', sep='')
print('\n"', report_df_savefile, 
      '"\n(Information on extracted reports and results pages)', sep='')
print('\n"', error_df_savefile, 
      '"\n(Information on reports and results pages which could not be extracted)', sep='')


Output filenames:

"Extracted PFAS data - "
(All PFAS sample results)

"Extracted lab report info - "
(Information on extracted reports and results pages)

"Error Reports - "
(Information on reports and results pages which could not be extracted)


In [ ]:

# Lookup datafile containing PFAS compound names, acronyms, and CAS numbers, for standardization and results screening
PFAS_names = pd.read_csv('Code/PFAS_compounds.csv')
PFAS_names = PFAS_names.loc[PFAS_names['PFAS18']==1,]


In [ ]:

# Scan 'Target Reports/' to identify folders to extract

folder_list = []
folder_list.append(glob.glob(target_path + '*/'))

folder_list = sum(folder_list, [])
folder_list.sort()
display(folder_list)


['Target Reports/2-0021045 - Stow - MA Fire Academy/']

In [ ]:

# create folder for extracted data files if it does not exist
# Will not overwrite existing folder data. 

try:
  os.mkdir(output_path)
  print("Created output folder: ", output_path)
except:
  print("Output folder already exists.")
  pass



Output folder already exists.


In [ ]:

# record start time 
t0 = datetime.datetime.now()


# Identify Folders & Report Files

### Scan "Target Reports/" folders and reports.
### Create a dataframe for storing filename and locations.
### Extract RTN info from filenames, if present. 

In [ ]:

# make dataframe of all filenames and paths from lab report folders
report_df = pd.DataFrame()

for f in folder_list:
    reports = glob.glob1(f, '*.pdf')

    f = f.strip(target_path)

    df = pd.DataFrame()
    df['report'] = reports
    df['folder'] = f + "/"
    report_df = report_df.append(df)

report_df = report_df.reset_index(drop=True)
print("Total reports:", report_df.shape[0])


Total reports: 42


In [ ]:

# extract RTN from filenames if present
for i,x in report_df.iterrows():
  
  source_attrs = {}
  RTN_regex = '\\d-\\d{7}'

  m = re.search(RTN_regex, x['folder'])
  if m is not None:
    source_attrs['folder_RTN'] = m.group()

  m = re.search(RTN_regex, x['report'])
  if m is not None:
    source_attrs['report_RTN'] = m.group()

  if 'folder_RTN' in source_attrs:
    report_df.loc[i, 'RTN'] = source_attrs['folder_RTN'] 

  if 'report_RTN' in source_attrs:
    report_df.loc[i, 'RTN'] = source_attrs['report_RTN'] 
    continue
  elif 'folder_RTN' in source_attrs:
    report_df.loc[i, 'RTN'] = source_attrs['folder_RTN'] 
    

In [ ]:

# setup dataframe columns for data to be added
warnings.filterwarnings("ignore")

report_df = report_df[['folder','report','RTN']]
report_df.loc[:,'address'] = ''
report_df.loc[:,'lab'] = ''
report_df.loc[:,'error'] = ''
report_df.loc[:,'results_page'] = ''
report_df.loc[:,'extracted'] = False

warnings.filterwarnings("default")


In [ ]:

# Summary Stats

print("\nTotal report files:", report_df.shape[0])

print('\nTotal reports labeled with RTNs:', report_df.loc[-pd.isna(report_df['RTN']),].shape[0])

print('\nTotal unique RTNs:', len(report_df['RTN'].unique()))




Total report files: 42

Total reports labeled with RTNs: 42

Total unique RTNs: 1


# Lab report identification
---
#### Identify all lab reports in folders.
#### Create table of report file info.
#### Determine and record testing lab for each report.
---



In [ ]:

# determine testing lab for each report

# suppress PyPDF2 extraction warnings
warnings.filterwarnings("ignore")

report_df['error'] = np.NaN

for i,x in report_df.iterrows():
    filename = target_path + x.folder + x.report

    if os.path.exists(filename):     
      try:
        report_type = utils.determine_report_type(filename)
      except:
        print(i, filename, ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,"error"] = 'Extraction function error'
        pass

    if 'lab' in report_type:
      report_df.loc[i, 'lab'] = report_type['lab']
      print(i, filename, " | ", report_type['lab'])
    elif 'error' in report_type:
      report_df.loc[i,"error"] = report_type['error']
      print(i, filename, ' | EXTRACTION ERROR:  ', report_type['error'])


# re-enable default warnings
warnings.filterwarnings("default")


0 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  sgs
1 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report L1945822 slope waste characterization 10-03-2019.pdf  |  alpha
2 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68136 stormwater_waste_wetland soil 09-17-2019.pdf  |  sgs
3 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68180 gas training area soil 09-18-2019.pdf  |  sgs
4 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68371 groundwater pfas 09-23-2019.pdf  |  sgs
5 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68135 white pond 09-17-2019.pdf  |  sgs
6 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68228 soil cmg4-7.pdf  |  sgs
7 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68229 surface water sw1.pdf  |  sgs
8 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68611 slope pfas 10-03-2019.pdf  |  sgs
9 Target 

In [ ]:

# Summary Stats

print("\nTotal report files:")
print(report_df.shape[0])

print("\nIdentified report files:")
print(report_df.loc[report_df['lab']!='',].shape[0])

print("\nUnidentified reports:")
print(report_df.loc[report_df['lab']=='',].shape[0])

print("\nLab Report types:")
print(report_df.loc[pd.isna(report_df['error']), 'lab'].value_counts())

print("\nUnique RTNs:")
print(len(report_df['RTN'].unique()))

if report_df['error'].count() > 0:
  print("\nErrors:")
  print(report_df['error'].value_counts())



Total report files:
42

Identified report files:
41

Unidentified reports:
1

Lab Report types:
alpha        19
sgs-final    11
sgs          11
Name: lab, dtype: int64

Unique RTNs:
1

Errors:
could not determine lab    1
Name: error, dtype: int64


# Lab Report Sample Results Page(s) Location
---
#### For all identified reports (without read errors):
#### Execute lab-specific functions to location results table in PFD and record page numbers in existing report file dataframe.
---



In [ ]:

# run through dataframe of lab reports (those without read errors)
# add a column with start/end pages for where the results sections appear in those reports

# suppress PyPDF2 extraction warnings
warnings.filterwarnings("ignore")

# for i,x in report_df.loc[308:309,].iterrows():
for i,x in report_df.loc[pd.isna(report_df['error']),].iterrows():
  filename = target_path + x.folder + x.report

  # print(i,x)

  results_pages = {}
  if os.path.exists(filename):     
    # For SGS Lab Reports
    if x['lab'] == 'sgs':
      try:
        results_pages = utils.SGS_find_results_pages(filename)
      except:
        print(i, filename, ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,'error'] = 'Data table extraction function error'
        pass
    # For CON_TEST Lab Reports
    if x['lab'] == 'con-test':
      try:
        results_pages = utils.CON_TEST_find_results_pages(filename)
      except:
        print(i, filename, ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,'error'] = 'Data table extraction function error'
        pass
    # For Alpha Lab Reports
    if x['lab'] == 'alpha':
      try:
        results_pages = utils.ALPHA_find_results_pages(filename)
      except:
        print(i, filename, ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,'error'] = 'Data table extraction function error'
        pass


    if 'results_pages' in results_pages:
      report_df.loc[i,'results_page'] = results_pages['results_pages']
      print(i, filename, " | ", x['lab'], " |", " results pages:", results_pages['results_pages'])    
    elif 'error' in results_pages:
      report_df.loc[i,'error'] = results_pages['error']
      print(i, filename, ' | EXTRACTION ERROR:  ', results_pages['error'])

# re-enable warnings
warnings.filterwarnings('default')


0 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  sgs  |  results pages: 7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
1 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report L1945822 slope waste characterization 10-03-2019.pdf  | EXTRACTION ERROR:   no results pages found
2 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68136 stormwater_waste_wetland soil 09-17-2019.pdf  |  sgs  |  results pages: 10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
3 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68180 gas training area soil 09-18-2019.pdf  |  sgs  |  results pages: 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
4 Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68371 groundwater pfas 09-23-2019.pdf  |  sgs  |  results pages: 8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
5 Target Reports/2-0021045 

In [ ]:

# Summary Stats

print("\nTotal report files:")
print(report_df.shape[0])

print("\nReports with identified sample results pages")
print(report_df.loc[report_df['results_page']!='',].shape[0])

print("\nReports without identified sample results pages")
print(report_df.loc[report_df['results_page']=='',].shape[0])

if report_df['error'].count() > 0:
  print("\nErrors:")
  print(report_df.value_counts('error', ascending=False))



Total report files:
42

Reports with identified sample results pages
28

Reports without identified sample results pages
14

Errors:
error
no results pages found     2
could not determine lab    1
dtype: int64


# Lab Report Sample Results Data Extraction
---
#### For all identified reports with results pages identified:

#### Execute lab-specific functions for data table extraction on all identified pages.
#### Process data columns for consistent naming and formats.
#### Obtain standardized PFAS compound names, acronyms and CAS numbers for each table.
#### Append all to master dataframe and export.  

---



In [ ]:

# explode lists of result pages to separate rows
report_df['results_page'] = report_df['results_page'].str.split(',')
report_df = report_df.explode('results_page')
report_df.reset_index(inplace=True)


In [ ]:

# suppress PyPDF2 extraction warnings
warnings.filterwarnings("ignore")

full_extracted_df = pd.DataFrame()

total_rows = report_df.loc[(-pd.isna(report_df['results_page'])) & (pd.isna(report_df['error'])),].shape[0] 
row_num = 0
last_report = ''

for i,x in report_df.loc[(-pd.isna(report_df['results_page'])) & (pd.isna(report_df['error'])),].iterrows():

  row_num += 1 
  filename = target_path + x.folder + x.report

  if os.path.exists(filename):  

    if filename != last_report:
      print()
      last_report = filename

    extraction_results = {}  
    extracted_df = pd.DataFrame()

    # For SGS Lab Reports
    if x['lab'] == 'sgs':
      try:
        extraction_results = utils.SGS_extract_results(filename, x['results_page'], PFAS_names)
      except:
        print(row_num, 'of', total_rows, ' | ', filename, ' |  lab:', x.lab,' |  page:', x['results_page'], ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,"error"] = 'Data table extraction function error'
        pass
    # For Con-Test Lab Reports
    if x['lab'] == 'con-test':
      try:
        extraction_results = utils.CON_TEST_extract_results(filename, x['results_page'], PFAS_names)
      except:
        print(row_num, 'of', total_rows, ' | ',  filename, ' |  lab:', x.lab,' |  page:', x['results_page'], ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,"error"] = 'Data table extraction function error'
        pass
    # For Alpha Lab Reports
    if x['lab'] == 'alpha':
      try:
        extraction_results = utils.ALPHA_extract_results(filename, x['results_page'], PFAS_names)
      except:
        print(row_num, 'of', total_rows, ' | ',  filename, ' |  lab:', x.lab,' |  page:', x['results_page'], ' | EXTRACTION ERROR: - no error info returned')
        report_df.loc[i,"error"] = 'Data table extraction function error'
        pass


    if 'results' in extraction_results:
      if not extraction_results['results'].empty:

        extracted_df = extraction_results['results']

        report_df.loc[i, 'extracted'] = True
        report_df.loc[i, 'extracted_compounds'] = extracted_df.shape[0]

        extracted_df['folder'] = x['folder']
        extracted_df['report'] = x['report']
        extracted_df['error'] = x['error']
        extracted_df['type'] = x['lab']
        extracted_df['page'] = x['results_page']

        if 'address' in extraction_results['results'].columns:
          report_df.loc[i, 'address'] = extraction_results['results']['address'].values[0]    
             
        if 'RTN' in x:
          extracted_df['RTN'] = x['RTN']

        if 'disposal_site_name' in x:
          extracted_df['disposal_site_name'] = x['disposal_site_name']

        if 'document_ID' in x:
          extracted_df['document_ID'] = x['document_ID']

        full_extracted_df = full_extracted_df.append(extracted_df, ignore_index=True)

        print(row_num, 'of', total_rows, ' | ',  filename, ' |  lab:', x.lab,' |  page:', x['results_page'], ' | Extracted PFAS compounds:', extracted_df.shape[0])

    elif 'error' in extraction_results:
      print(row_num, 'of', total_rows, ' | ',  filename, ' |  lab:', x.lab,' |  page:', x['results_page'], ' | EXTRACTION ERROR:', extraction_results['error'])
      report_df.loc[i,"error"] = extraction_results['error']
      report_df.loc[i,"extracted"] = False


if 'address' not in full_extracted_df.columns:
  full_extracted_df['address'] = ''   

# re-enable warnings
warnings.filterwarnings("default")




1 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 7  | Extracted PFAS compounds: 16
2 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 8  | EXTRACTION ERROR: not sample results data
3 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 9  | Extracted PFAS compounds: 16
4 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 10  | EXTRACTION ERROR: not sample results data
5 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 11  | Extracted PFAS compounds: 16
6 of 246  |  Target Reports/2-0021045 - Stow - MA Fire Academy/lab report FA68181 soil borings 09-18-2019.pdf  |  lab: sgs  |  page: 12 

In [ ]:

# Update dataframe column ordering

report_df = report_df[['folder','report','RTN', 'address', 
                       'lab','error',
                       'results_page','extracted', 'extracted_compounds']]

full_extracted_df = full_extracted_df[['folder','report','RTN',
                                       'address', 'date_sampled','sample_id',
                                       'lab','page',
                                       'Matrix','DF','Acronym','Compound','CAS','Result','RL','MCL','MDL','Units']]



In [ ]:

# Isolate all report pages that received error flags during the extraction process

error_df = report_df.loc[report_df['extracted']!=True,]
error_df = error_df.drop(['extracted','extracted_compounds'], axis=1)

# Isolate all successfully extracted report pages
extracted_report_df = report_df.loc[report_df['extracted'],]
extracted_report_df = extracted_report_df.drop(['extracted', 'error'], axis=1)



In [ ]:

# Summary Stats

print("\nTotal report files:")
print(len(extracted_report_df['report'].unique()))

print("\nTotal identified results pages to extract:")
print(extracted_report_df.loc[-pd.isna(extracted_report_df['results_page']),].shape[0])

print("\nTotal results pages extracted:")
print(extracted_report_df.shape[0])

print("\nPercent of results pages extracted:")
print(np.round(extracted_report_df.shape[0] / report_df.shape[0],2)*100)

print("\nExtracted pages by lab type:")
print(extracted_report_df.value_counts('lab'))

print("\nTotal extracted compounds:")
print(full_extracted_df.shape[0])

print("\nExtracted compound totals:")
print(full_extracted_df.value_counts('Acronym', ascending=False))

print("\nErrors by type:")
print(error_df['error'].value_counts(ascending=False))

print("\nErrors by lab type:")
print(error_df.loc[-pd.isna(error_df['error']),].value_counts('lab'))

print("\nErrors by folder:")
print(error_df.loc[-pd.isna(error_df['error']),].value_counts('folder'))



Total report files:
28

Total identified results pages to extract:
161

Total results pages extracted:
161

Percent of results pages extracted:
65.0

Extracted pages by lab type:
lab
sgs      84
alpha    77
dtype: int64

Total extracted compounds:
2657

Extracted compound totals:
Acronym
PFOA             161
PFTA             159
PFUnA            158
PFHpA            158
PFBS             158
PFTrDA           158
PFDoA            158
PFDA             158
PFHxA            158
PFHxS            158
PFNA             158
PFOS             158
ADONA            147
9Cl-PF 3ONS      146
N-MeFOSAA         84
N-EtFOSAA         84
NMeFOSAA          74
NEtFOSAA          74
HFPO-DA           74
11Cl-PF 3OUdS     74
dtype: int64

Errors by type:
not sample results data    74
no results pages found      2
could not determine lab     1
Name: error, dtype: int64

Errors by lab type:
lab
sgs      74
alpha     2
          1
dtype: int64

Errors by folder:
folder
2-0021045 - Stow - MA Fire Academy/    77
dt

#Finalize and Save Output  
---
###Convert and save any detected result values to a numeric format.
###Identify a detection limit to use for each sample result, for imputing ND values. 
###Add date and time to output filenames.
###Save data.  
---



In [ ]:

# Notes: 

# ND data does not necessarily mean that PFAS amounts are zero for a sample, but only that they are not above the lab's detectable limit.
# Instead of dropping these cases, various methods can be used for imputing ND results.
# Typically, ND data can be imputed with 1/2 the detection limit for that compound sample test. 
# 1/2 MDL would be the most appropriate way to impute ND using the 1/2 method.  
# If MDL is not available, 1/2 RL is appropriate. 
# More elaborate statistical methods are often used, such as Regression over Ordered Statistics (ROS). 
# An Imputation Pipeline .ipynb script is provided that performs either method, and may be utilized as desired.


In [ ]:

# Set up results columns for imputing Non-Detect (ND) values during later processing

full_extracted_df.loc[full_extracted_df['Result']=='ND', 'Result_val'] = np.NaN 
full_extracted_df['Result_val'] = pd.to_numeric(full_extracted_df['Result'], errors='coerce')
full_extracted_df['RL'] = pd.to_numeric(full_extracted_df['RL'],errors='coerce')
full_extracted_df['MDL'] = pd.to_numeric(full_extracted_df['MDL'],errors='coerce')

# if MDL is available, use this as the detection limit for ND imputation, otherwise use RL
full_extracted_df.loc[-pd.isna(full_extracted_df['MDL']),'limit'] = full_extracted_df['MDL']
full_extracted_df.loc[pd.isna(full_extracted_df['MDL']),'limit'] = full_extracted_df['RL']


In [ ]:

# Add date and time to output filenames

tz = pytz.timezone('America/New_York') 
timestamp = datetime.datetime.now(tz)
timestamp_str = timestamp.strftime("%m-%d-%Y %H:%M")

extracted_data_savefile = extracted_data_savefile + timestamp_str + '.csv'
report_df_savefile = report_df_savefile + timestamp_str + '.csv'
error_df_savefile = error_df_savefile + timestamp_str + '.csv'

print('Output filenames:')
print('\n"', extracted_data_savefile, 
      '"\n(All PFAS sample results)', sep='')
print('\n"', report_df_savefile, 
      '"\n(Information on extracted reports and results pages)', sep='')
print('\n"', error_df_savefile, 
      '"\n(Information on reports and results pages which could not be extracted)', sep='')



Output filenames:

"Extracted PFAS data - 12-16-2021 08:07.csv"
(All PFAS sample results)

"Extracted lab report info - 12-16-2021 08:07.csv"
(Information on extracted reports and results pages)

"Error Reports - 12-16-2021 08:07.csv"
(Information on reports and results pages which could not be extracted)


In [ ]:

# Save data to output folder
full_extracted_df.to_csv(output_path + extracted_data_savefile)
extracted_report_df.to_csv(output_path + report_df_savefile)
error_df.to_csv(output_path + error_df_savefile)
  

In [ ]:

# Total runtime

t1 = datetime.datetime.now()
print('Total Runtime (h:m:s.ms):', t1 - t0)



Total Runtime (h:m:s.ms): 0:10:29.398208


#End